In [6]:
#IMPORT LIBRARY AND API_KEY
from dotenv import load_dotenv
import os
import requests
import pprint
import json
import pandas as pd
from datetime import datetime

load_dotenv("API_KEY.env")

GROQ_API_KEY = os.getenv("GROQ_API_KEY")
SECTORS_API_KEY = os.getenv("SECTORS_API_KEY")

In [30]:
#CREATING THE PROMPT AND TOOLS
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_groq import ChatGroq
from langchain.agents import create_tool_calling_agent, AgentExecutor

def retrieve_from_endpoint(url: str) ->pd.DataFrame:
    headers = {"Authorization": SECTORS_API_KEY}

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        data = response.json()
    except requests.exceptions.HTTPError as err:
        raise SystemExit(err)
    df = pd.DataFrame(data)
    return df

@tool
def get_top_companies_by_tx_volume(
    start_date: str, end_date: str, top_n: int = 5
) -> str:
    """
    Get top companies by transaction volume
    """
    url = f"https://api.sectors.app/v1/most-traded/?start={start_date}&end={end_date}&n_stock={top_n}"

    return retrieve_from_endpoint(url)

tools = [get_top_companies_by_tx_volume]
llm = ChatGroq(
    temperature=0,
    model_name="llama3-groq-70b-8192-tool-use-preview",
    groq_api_key=GROQ_API_KEY,
)

current_date = datetime.now().strftime('%Y-%m-%d')

prompt_text = f"""
Answer the following queries as factually and analytically as possible. 
If start and end dates are not explicitly provided, infer them from the query.  
Whenever you return a list of names, include the corresponding values for each name. 
If the volume is for a single day, set the start and end dates to be the same.

Today's date is {current_date}.
"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", prompt_text),
        ("human", "{input}"),
        # msg containing previous agent tool invocations and corresponding tool outputs
        MessagesPlaceholder("agent_scratchpad"),
    ]
)
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


In [32]:
#CREATING THE QUERIES
queries = [
    "What are the top 5 companies by transaction volume on the first of this month?"
    "What are the most traded stock yesterday?"
    "What are the top 7 most traded stocks between 6th June to 10th June this year?"
]

for query in queries:
    print("Question:", query)
    result = agent_executor.invoke({"input": query})
    print("Answer:", "\n", result["output"], "\n\n======\n\n")

Question: What are the top 5 companies by transaction volume on the first of this month?What are the most traded stock yesterday?What are the top 7 most traded stocks between 6th June to 10th June this year?


> Entering new AgentExecutor chain...

Invoking: `get_top_companies_by_tx_volume` with `{'start_date': '2024-08-01', 'end_date': '2024-08-01', 'top_n': 5}`


                                          2024-08-01
0  {'symbol': 'GOTO.JK', 'company_name': 'PT GoTo...
1  {'symbol': 'BIPI.JK', 'company_name': 'PT Astr...
2  {'symbol': 'NICL.JK', 'company_name': 'PT PAM ...
3  {'symbol': 'WIKA.JK', 'company_name': 'PT Wija...
4  {'symbol': 'DEWA.JK', 'company_name': 'Darma H...
Invoking: `get_top_companies_by_tx_volume` with `{'start_date': '2024-08-10', 'end_date': '2024-08-10', 'top_n': 1}`


Empty DataFrame
Columns: []
Index: []
Invoking: `get_top_companies_by_tx_volume` with `{'start_date': '2024-06-06', 'end_date': '2024-06-10', 'top_n': 7}`


                                      